In [1]:
tokenizer_name = 'bert-base-uncased'

In [2]:
import numpy as np

In [3]:
import pandas as pd
df = pd.read_csv('outpt.csv')

In [4]:
df.head()

,sentences,labels
0,"['Intravenous', 'azithromycin', '-', 'induced'...","['O', 'B-drug', 'O', 'O', 'B-effect', 'O']"
1,"['Immobilization', ',', 'while', ""Paget's"", 'b...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"['Unaccountable', 'severe', 'hypercalcemia', '...","['O', 'O', 'B-effect', 'O', 'O', 'O', 'O', 'O'..."
3,"['METHODS', ':', 'We', 'report', 'two', 'cases...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect'..."
4,"['METHODS', ':', 'We', 'report', 'two', 'cases...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect'..."


In [5]:
def ret_list(text):
    return eval(text)
df['sentences'] = df['sentences'].apply(ret_list)
df['labels'] = df['labels'].apply(ret_list)

In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.15, random_state=42)

In [7]:
len(train), len(test)

(5871, 1037)

In [8]:
sentences = train['sentences'].to_list()
labels = train['labels'].to_list()

In [9]:
from transformers import BertTokenizer
import numpy as np
tokenizer = BertTokenizer.from_pretrained(tokenizer_name, do_lower_case=True)

In [10]:
TokenLength = [len(tokenizer.encode(' '.join(i),add_special_tokens=True)) for i in sentences]

In [11]:
print("Minimum length: {:,} tokens".format(min(TokenLength)))
print("Maximum length: {:,} tokens".format(max(TokenLength)))
print("Average length: {:,} tokens".format(int(np.median(TokenLength))))

Minimum length: 7 tokens
Maximum length: 119 tokens
Average length: 35 tokens


In [12]:
unique_labels = set()
for label in labels:
    for l in label:
        unique_labels.add(l)
        
unique_labels 

{'B-drug', 'B-effect', 'I-drug', 'I-effect', 'O'}

In [13]:
data_tags = list(unique_labels)
data_tags

['O', 'I-drug', 'I-effect', 'B-effect', 'B-drug']

In [14]:
tag_values = data_tags
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}
tag2idx

{'O': 0, 'I-drug': 1, 'I-effect': 2, 'B-effect': 3, 'B-drug': 4, 'PAD': 5}

In [15]:
print(sentences[0])
print(labels[0])

['However', ',', 'peripheral', 'neuropathy', 'and', 'bone', 'marrow', 'depression', 'led', 'to', 'linezolid', 'withdrawal', 'in', 'seven', 'patients', ',', 'and', 'neuropathy', 'may', 'not', 'be', 'fully', 'reversible', 'in', 'all', 'patients', '.']
['O', 'O', 'O', 'B-effect', 'O', 'O', 'O', 'O', 'O', 'O', 'B-drug', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [16]:
def tokenize_and_align_labels(sentences, labels):
    tokenized_sentences = []
    aligned_labels = []
    for word, label in zip(sentences, labels):
        tokenized_word = tokenizer.tokenize(word)
        subwords = len(tokenized_word)
        
        tokenized_sentences.extend(tokenized_word)
        
        if subwords > 1 and label != "O":
            c = label.replace("B-","I-")
            aligned_labels.extend([label] + [c] * (subwords - 1))
        else:
            aligned_labels.extend([label]*subwords)
          
    return tokenized_sentences, aligned_labels

In [17]:
tokenized_data = [tokenize_and_align_labels(sent, label) for sent, label in zip(sentences, labels)]

In [18]:
print(tokenized_data[0])

(['however', ',', 'peripheral', 'ne', '##uro', '##pathy', 'and', 'bone', 'marrow', 'depression', 'led', 'to', 'line', '##zo', '##lid', 'withdrawal', 'in', 'seven', 'patients', ',', 'and', 'ne', '##uro', '##pathy', 'may', 'not', 'be', 'fully', 'rev', '##ers', '##ible', 'in', 'all', 'patients', '.'], ['O', 'O', 'O', 'B-effect', 'I-effect', 'I-effect', 'O', 'O', 'O', 'O', 'O', 'O', 'B-drug', 'I-drug', 'I-drug', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect', 'I-effect', 'I-effect', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])


In [19]:
tokenized_texts = [sentandlabel[0] for sentandlabel in tokenized_data]
new_labels = [sentandlabel[1] for sentandlabel in tokenized_data]

In [20]:
print(tokenized_texts[0])
print(new_labels[0])

['however', ',', 'peripheral', 'ne', '##uro', '##pathy', 'and', 'bone', 'marrow', 'depression', 'led', 'to', 'line', '##zo', '##lid', 'withdrawal', 'in', 'seven', 'patients', ',', 'and', 'ne', '##uro', '##pathy', 'may', 'not', 'be', 'fully', 'rev', '##ers', '##ible', 'in', 'all', 'patients', '.']
['O', 'O', 'O', 'B-effect', 'I-effect', 'I-effect', 'O', 'O', 'O', 'O', 'O', 'O', 'B-drug', 'I-drug', 'I-drug', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect', 'I-effect', 'I-effect', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [24]:
!pip show tensorflow

In [21]:
import keras

ImportError: cannot import name 'app' from 'absl' (unknown location)

In [26]:
#!pip install keras

In [27]:
#!pip install tensorflow

In [22]:
from keras.utils import pad_sequences

ImportError: cannot import name 'app' from 'absl' (unknown location)

In [34]:
# !pip install --upgrade tensorflow


In [35]:
# !pip install --upgrade keras

In [38]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(text) for text in tokenized_texts],
                          maxlen=130, dtype="long", value=0.0, 
                          truncating="post", padding="post")
label_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in new_labels],maxlen=130, value=tag2idx["PAD"], padding="post",dtype="long", truncating="post")

In [39]:
print(input_ids[0])
print(label_tags[0])

[ 2174  1010 15965 11265 10976 20166  1998  5923 24960  6245  2419  2000
  2240  6844 21273 10534  1999  2698  5022  1010  1998 11265 10976 20166
  2089  2025  2022  3929  7065  2545  7028  1999  2035  5022  1012     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0]
[2 2 2 1 4 4 2 2 2 2 2 2 0 3 3 2 2 2 2 2 2 1 4 4 2 2 2 2 2 2 2 2 2 2 2 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5

In [40]:
print(len(input_ids[0]))
print(len(label_tags[0]))

130
130
